In [49]:
# !pip install tensorflow

In [50]:
#import librabries
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from datetime import datetime

In [51]:
risk_purchase_data = pd.read_csv('/content/risk-dataset.txt', delimiter="\t")

In [52]:
risk_purchase_data.head()

,ORDER_ID,CLASS,B_EMAIL,B_TELEFON,B_BIRTHDATE,FLAG_LRIDENTISCH,FLAG_NEWSLETTER,Z_METHODE,Z_CARD_ART,Z_CARD_VALID,...,FAIL_RPLZ,FAIL_RORT,FAIL_RPLZORTMATCH,SESSION_TIME,NEUKUNDE,AMOUNT_ORDER_PRE,VALUE_ORDER_PRE,DATE_LORDER,MAHN_AKT,MAHN_HOECHST
0,49917,no,yes,no,1/17/1973,yes,yes,check,?,5.2006,...,no,no,no,8,yes,0,0.00,?,?,?
1,49919,no,yes,yes,12/8/1970,no,no,credit_card,Visa,12.2007,...,yes,no,no,13,yes,0,0.00,?,?,?
2,49923,no,yes,no,4/3/1972,yes,no,check,?,12.2007,...,no,no,no,3,yes,0,0.00,?,?,?
3,49924,no,no,yes,8/1/1966,yes,no,check,?,1.2007,...,no,no,no,11,no,4,75.72,5/12/2002,0,0
4,49927,no,yes,yes,12/21/1969,yes,no,credit_card,Eurocard,12.2006,...,no,no,no,16,yes,0,0.00,?,?,?


In [53]:
risk_purchase_data.columns

Index(['ORDER_ID', 'CLASS', 'B_EMAIL', 'B_TELEFON', 'B_BIRTHDATE',
       'FLAG_LRIDENTISCH', 'FLAG_NEWSLETTER', 'Z_METHODE', 'Z_CARD_ART',
       'Z_CARD_VALID', 'Z_LAST_NAME', 'VALUE_ORDER', 'WEEKDAY_ORDER',
       'TIME_ORDER', 'AMOUNT_ORDER', 'ANUMMER_01', 'ANUMMER_02', 'ANUMMER_03',
       'ANUMMER_04', 'ANUMMER_05', 'ANUMMER_06', 'ANUMMER_07', 'ANUMMER_08',
       'ANUMMER_09', 'ANUMMER_10', 'CHK_LADR', 'CHK_RADR', 'CHK_KTO',
       'CHK_CARD', 'CHK_COOKIE', 'CHK_IP', 'FAIL_LPLZ', 'FAIL_LORT',
       'FAIL_LPLZORTMATCH', 'FAIL_RPLZ', 'FAIL_RORT', 'FAIL_RPLZORTMATCH',
       'SESSION_TIME', 'NEUKUNDE', 'AMOUNT_ORDER_PRE', 'VALUE_ORDER_PRE',
       'DATE_LORDER', 'MAHN_AKT', 'MAHN_HOECHST'],
      dtype='object')

In [54]:
risk_purchase_data.shape

(30000, 44)

In [55]:
risk_purchase_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ORDER_ID           30000 non-null  int64  
 1   CLASS              30000 non-null  object 
 2   B_EMAIL            30000 non-null  object 
 3   B_TELEFON          30000 non-null  object 
 4   B_BIRTHDATE        30000 non-null  object 
 5   FLAG_LRIDENTISCH   30000 non-null  object 
 6   FLAG_NEWSLETTER    30000 non-null  object 
 7   Z_METHODE          30000 non-null  object 
 8   Z_CARD_ART         30000 non-null  object 
 9   Z_CARD_VALID       30000 non-null  float64
 10  Z_LAST_NAME        30000 non-null  object 
 11  VALUE_ORDER        30000 non-null  float64
 12  WEEKDAY_ORDER      30000 non-null  object 
 13  TIME_ORDER         30000 non-null  object 
 14  AMOUNT_ORDER       30000 non-null  int64  
 15  ANUMMER_01         30000 non-null  int64  
 16  ANUMMER_02         300

In [56]:
risk_purchase_data.replace('?', np.nan, inplace=True)

<ipython-input-56-4f8e6bd467b4>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  risk_purchase_data.replace('?', np.nan, inplace=True)


In [57]:
risk_purchase_data.isnull().sum()


,0
ORDER_ID,0
CLASS,0
B_EMAIL,0
B_TELEFON,0
B_BIRTHDATE,2942
FLAG_LRIDENTISCH,0
FLAG_NEWSLETTER,0
Z_METHODE,0
Z_CARD_ART,18654
Z_CARD_VALID,0


In [60]:
def calculate_age(birthdate):
    if pd.isnull(birthdate):
        return 0
    today = datetime.today()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

risk_purchase_data['B_BIRTHDATE'] = pd.to_datetime(risk_purchase_data['B_BIRTHDATE'], errors='coerce')

# Apply function to Birthdate column
risk_purchase_data['Age'] = risk_purchase_data['B_BIRTHDATE'].apply(calculate_age)

,ORDER_ID,CLASS,B_EMAIL,B_TELEFON,B_BIRTHDATE,FLAG_LRIDENTISCH,FLAG_NEWSLETTER,Z_METHODE,Z_CARD_ART,Z_CARD_VALID,...,FAIL_RORT,FAIL_RPLZORTMATCH,SESSION_TIME,NEUKUNDE,AMOUNT_ORDER_PRE,VALUE_ORDER_PRE,DATE_LORDER,MAHN_AKT,MAHN_HOECHST,Age
0,49917,no,yes,no,1973-01-17,yes,yes,check,NaN,5.2006,...,no,no,8,yes,0,0.00,NaN,NaN,NaN,52
1,49919,no,yes,yes,1970-12-08,no,no,credit_card,Visa,12.2007,...,no,no,13,yes,0,0.00,NaN,NaN,NaN,54
2,49923,no,yes,no,1972-04-03,yes,no,check,NaN,12.2007,...,no,no,3,yes,0,0.00,NaN,NaN,NaN,52
3,49924,no,no,yes,1966-08-01,yes,no,check,NaN,1.2007,...,no,no,11,no,4,75.72,5/12/2002,0,0,58
4,49927,no,yes,yes,1969-12-21,yes,no,credit_card,Eurocard,12.2006,...,no,no,16,yes,0,0.00,NaN,NaN,NaN,55


In [62]:
risk_purchase_data.drop(['ORDER_ID','B_BIRTHDATE'], axis=1, inplace=True)

In [ ]:
X = risk_purchase_data.drop(columns=["CLASS"])
y = risk_purchase_data["CLASS"]

In [ ]:
y = LabelEncoder().fit_transform(y)

In [ ]:
X.head()